# 14-813/18-813 Course Project – Option 2

### Task-I: Build and populate necessary tables

Firstly, we contruct the spark section.

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 13:37:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In the following, we import the MQTT training and test dataset and add a new column to identify them.

In [2]:
from pyspark import SparkFiles
from pyspark.sql.functions import lit


col_names = [
    "tcp.flags",
    "tcp.time_delta",
    "tcp.len",
    "mqtt.conack.flags",
    "mqtt.conack.flags.reserved",
    "mqtt.conack.flags.sp",
    "mqtt.conack.val",
    "mqtt.conflag.cleansess",
    "mqtt.conflag.passwd",
    "mqtt.conflag.qos",
    "mqtt.conflag.reserved",
    "mqtt.conflag.retain",
    "mqtt.conflag.uname",
    "mqtt.conflag.willflag",
    "mqtt.conflags",
    "mqtt.dupflag",
    "mqtt.hdrflags",
    "mqtt.kalive",
    "mqtt.len",
    "mqtt.msg",
    "mqtt.msgid",
    "mqtt.msgtype",
    "mqtt.proto_len",
    "mqtt.protoname",
    "mqtt.qos",
    "mqtt.retain",
    "mqtt.sub.qos",
    "mqtt.suback.qos",
    "mqtt.ver",
    "mqtt.willmsg",
    "mqtt.willmsg_len",
    "mqtt.willtopic",
    "mqtt.willtopic_len",
    "target"
]


# train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_augmented.csv"
# test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_augmented.csv"
train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_reduced.csv"
test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_reduced.csv"

df_train = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)

In [3]:
df_train = df_train.withColumn("dataset", lit(0))
df_test = df_test.withColumn("dataset", lit(1))

df_train.show(1, vertical=True)
df_test.show(1, vertical=True)

print(df_train.count())
print(df_test.count())

22/11/26 13:37:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0--------------------------------
 tcp.flags                  | 0x00000018 
 tcp.time_delta             | 0.998867   
 tcp.len                    | 10         
 mqtt.conack.flags          | 0          
 mqtt.conack.flags.reserved | 0.0        
 mqtt.conack.flags.sp       | 0.0        
 mqtt.conack.val            | 0.0        
 mqtt.conflag.cleansess     | 0.0        
 mqtt.conflag.passwd        | 0.0        
 mqtt.conflag.qos           | 0.0        
 mqtt.conflag.reserved      | 0.0        
 mqtt.conflag.retain        | 0.0        
 mqtt.conflag.uname         | 0.0        
 mqtt.conflag.willflag      | 0.0        
 mqtt.conflags              | 0          
 mqtt.dupflag               | 0.0        
 mqtt.hdrflags              | 0x00000030 
 mqtt.kalive                | 0.0        
 mqtt.len             

In [4]:
df_train.printSchema()
df_test.printSchema()

root
 |-- tcp.flags: string (nullable = true)
 |-- tcp.time_delta: double (nullable = true)
 |-- tcp.len: integer (nullable = true)
 |-- mqtt.conack.flags: string (nullable = true)
 |-- mqtt.conack.flags.reserved: double (nullable = true)
 |-- mqtt.conack.flags.sp: double (nullable = true)
 |-- mqtt.conack.val: double (nullable = true)
 |-- mqtt.conflag.cleansess: double (nullable = true)
 |-- mqtt.conflag.passwd: double (nullable = true)
 |-- mqtt.conflag.qos: double (nullable = true)
 |-- mqtt.conflag.reserved: double (nullable = true)
 |-- mqtt.conflag.retain: double (nullable = true)
 |-- mqtt.conflag.uname: double (nullable = true)
 |-- mqtt.conflag.willflag: double (nullable = true)
 |-- mqtt.conflags: string (nullable = true)
 |-- mqtt.dupflag: double (nullable = true)
 |-- mqtt.hdrflags: string (nullable = true)
 |-- mqtt.kalive: double (nullable = true)
 |-- mqtt.len: double (nullable = true)
 |-- mqtt.msg: string (nullable = true)
 |-- mqtt.msgid: double (nullable = true)
 |-

Before injesting the date (which will actually create the mqtt table), we need to revise the metadata and ensure certain constraints (data type of columns), as it mentioned in the handout.

Here we rename the columns according to programming conventions.

In [5]:
new_cols = [
    '_'.join(item.split('.')) for item in col_names[:-1]
]

renamed_columns_df = df_train.union(df_test)    # concatenate train & test
for i in range(len(new_cols)):
    renamed_columns_df = renamed_columns_df.withColumnRenamed(col_names[i], new_cols[i])
    df_train = df_train.withColumnRenamed(col_names[i], new_cols[i])
    df_test = df_test.withColumnRenamed(col_names[i], new_cols[i])

print(renamed_columns_df.count())
renamed_columns_df.printSchema()

330936
root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = tr

Then we determine the data type of each columns, depending on the data it contains and the meaning it carries.

However, we discovered that the data is quite clean in terms of the data type. Most of the data is in <b>double</b> type or integer type, which is already good numerical representation.

The following are the existing string columns.
```
|-- tcp_flags: string (nullable = true)
|-- mqtt_conack_flags: string (nullable = true)
|-- mqtt_conflags: string (nullable = true)
|-- mqtt_hdrflags: string (nullable = true)
|-- mqtt_msg: string (nullable = true)
|-- mqtt_protoname: string (nullable = true)
|-- target: string (nullable = true)
```

For the <b>flags</b>, we considered about changing them into integer, since they can be represented in integer at this stage. However, it is more reasonable to convert into one-hot encoding later when doing data engineering, since they are nominal type. Same intuition can be applied to <b>target</b> feature.

For <b>mqtt_msg</b>, it is a piece of message without specific form, so it should be kept in string type. And as I inspect <b>mqtt_protoname</b>, it appears to be another nominal feature with two possible labels. 

<!-- Here we changed the type of two flags since according to the paper and protocals, the flags will only take several fixed distince values, which are represented by flipping the ending bits of a integer size number. Therefore, it can always be represented in terms of a equivalent integer.

#### TODO: but this is not ordinal, this should be nominal -->

In [6]:
import pyspark.sql.functions as F

casted_types_df = renamed_columns_df
casted_types_df.select("tcp_flags").distinct().show(vertical=True)
casted_types_df.select("mqtt_conack_flags").distinct().show(vertical=True)
casted_types_df.select("mqtt_conflags").distinct().show(vertical=True)
casted_types_df.select("mqtt_hdrflags").distinct().show(vertical=True)
casted_types_df.select("mqtt_msg").distinct().show(vertical=True)
casted_types_df.select("mqtt_protoname").distinct().show(vertical=True)
casted_types_df.select("mqtt_protoname").groupBy("mqtt_protoname").count().show()
casted_types_df.select("target").show(vertical=True)

# casted_types_df = renamed_columns_df.withColumn(
#     "new_tcp_flags", 
#     F.expr("conv(substring(tcp_flags, 3), 16, 10)")
# ).drop("tcp_flags").distinct()      # deleting duplicate rows

-RECORD 0---------------
 tcp_flags | 0x00000004 
-RECORD 1---------------
 tcp_flags | 0x00000010 
-RECORD 2---------------
 tcp_flags | 0x00000019 
-RECORD 3---------------
 tcp_flags | 0x00000002 
-RECORD 4---------------
 tcp_flags | 0x00000012 
-RECORD 5---------------
 tcp_flags | 0x00000014 
-RECORD 6---------------
 tcp_flags | 0x00000018 
-RECORD 7---------------
 tcp_flags | 0x00000011 

-RECORD 0-----------------------
 mqtt_conack_flags | 0          
-RECORD 1-----------------------
 mqtt_conack_flags | 0x00000000 

-RECORD 0-------------------
 mqtt_conflags | 0          
-RECORD 1-------------------
 mqtt_conflags | 0x00000002 
-RECORD 2-------------------
 mqtt_conflags | 0x000000c2 
-RECORD 3-------------------
 mqtt_conflags | 0x00000082 

-RECORD 0-------------------
 mqtt_hdrflags | 0x00000031 
-RECORD 1-------------------
 mqtt_hdrflags | 0x00000090 
-RECORD 2-------------------
 mqtt_hdrflags | 0          
-RECORD 3-------------------
 mqtt_hdrflags | 0x000000d0 
-

-RECORD 0------------------------
 mqtt_msg | 31393430433266463... 
-RECORD 1------------------------
 mqtt_msg | 45424461623163656... 
-RECORD 2------------------------
 mqtt_msg | 39643563323231333... 
-RECORD 3------------------------
 mqtt_msg | 62384444614263623... 
-RECORD 4------------------------
 mqtt_msg | 63304463463044326... 
-RECORD 5------------------------
 mqtt_msg | 323939.0             
-RECORD 6------------------------
 mqtt_msg | 43356663664239353... 
-RECORD 7------------------------
 mqtt_msg | 32336632314441453... 
-RECORD 8------------------------
 mqtt_msg | 37444243394641443... 
-RECORD 9------------------------
 mqtt_msg | 32433732343042634... 
-RECORD 10-----------------------
 mqtt_msg | 39373045306642463... 
-RECORD 11-----------------------
 mqtt_msg | 35614162354646626... 
-RECORD 12-----------------------
 mqtt_msg | 36646133624637413... 
-RECORD 13-----------------------
 mqtt_msg | 333032               
-RECORD 14-----------------------
 mqtt_msg | 37

Write the data into postgre table through pyspark jdbc API and verify the result with read command.

In [9]:
db_properties={}
db_properties['username']="14813"
db_properties['password']="bigdata"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt.processed_data"


casted_types_df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

# df_test.write.format("jdbc")\
# .mode("append")\
# .option("url", db_properties['url'])\
# .option("dbtable", db_properties['table'])\
# .option("user", db_properties['username'])\
# .option("password", db_properties['password'])\
# .option("Driver", db_properties['driver'])\
# .save()


Py4JJavaError: An error occurred while calling o218.save.
: org.postgresql.util.PSQLException: ERROR: schema "mqtt" does not exist
  Position: 14
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2676)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2366)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:356)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:490)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:408)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:329)
	at org.postgresql.jdbc.PgStatement.executeCachedSql(PgStatement.java:315)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:291)
	at org.postgresql.jdbc.PgStatement.executeUpdate(PgStatement.java:265)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.executeStatement(JdbcUtils.scala:1060)
	at org.apache.spark.sql.execution.datasources.jdb

In [38]:

sqlContext = SQLContext(spark.sparkContext)
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)

/usr/local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


-RECORD 0--------------------------------
 tcp_flags                  | 0x00000010 
 tcp_time_delta             | 2.3E-5     
 tcp_len                    | 0          
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0          
 mqtt_kalive                | 0.0        
 mqtt_len                   | 0.0        
 mqtt_msg                   | 0          
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 0.0        
 mqtt_proto_len             | 0.0 

### Task-II: Conduct analytics on your dataset

In task 2, we conduct analysis. Firstly, the average length of mqtt message is 31.44.

In [51]:

import pyspark.sql.functions as F
 
# df_len = df_train.withColumn("mqtt_msg_len", F.length("mqtt_msg"))
df_train.agg({'mqtt_len': 'avg'}).show()

+------------------+
|     avg(mqtt_len)|
+------------------+
|31.435725201384873|
+------------------+

-RECORD 0------------------------------------------
 tcp_flags                  | 0x00000018           
 tcp_time_delta             | 0.998867             
 tcp_len                    | 10                   
 mqtt_conack_flags          | 0                    
 mqtt_conack_flags_reserved | 0.0                  
 mqtt_conack_flags_sp       | 0.0                  
 mqtt_conack_val            | 0.0                  
 mqtt_conflag_cleansess     | 0.0                  
 mqtt_conflag_passwd        | 0.0                  
 mqtt_conflag_qos           | 0.0                  
 mqtt_conflag_reserved      | 0.0                  
 mqtt_conflag_retain        | 0.0                  
 mqtt_conflag_uname         | 0.0                  
 mqtt_conflag_willflag      | 0.0                  
 mqtt_conflags              | 0                    
 mqtt_dupflag               | 0.0                  
 mqtt_hdrf

The average tcp length of each target group is given as the folloing.

In [53]:
from pyspark.sql.functions import sum,avg,max
df_target_len = df_train.groupBy("target").agg(avg("tcp_len").alias("tcp_len_avg"))
df_target_len.show()

+----------+------------------+
|    target|       tcp_len_avg|
+----------+------------------+
|   slowite| 3.463592609843192|
|bruteforce| 3.260886699507389|
|     flood|13479.687645687645|
| malformed|20.892362019356526|
|       dos| 305.3959037254816|
|legitimate| 7.785061817930654|
+----------+------------------+



In this block, we provide a function that would return the top X frequent target, for an X input by the user.

In [59]:
from pyspark.sql.functions import count

def showTop():
    top_x = int(input("please input an integer indicating the top number of items you want >>>"))
    df_tcp_flags = df_train.groupBy("tcp_flags").agg(count("tcp_flags").alias("tcp_flags_count")).sort("tcp_flags_count",ascending=False)
    df_tcp_flags.show(top_x, vertical=True)

showTop()


-RECORD 0---------------------
 tcp_flags       | 0x00000018 
 tcp_flags_count | 128896     
-RECORD 1---------------------
 tcp_flags       | 0x00000010 
 tcp_flags_count | 93061      
-RECORD 2---------------------
 tcp_flags       | 0x00000011 
 tcp_flags_count | 2932       
-RECORD 3---------------------
 tcp_flags       | 0x00000012 
 tcp_flags_count | 2564       
-RECORD 4---------------------
 tcp_flags       | 0x00000002 
 tcp_flags_count | 2564       
only showing top 5 rows



In this part, we explore the most popular target among the listed ones on twitter, with <i>tweepy</i> and <i>kafka</i>.

In this file, only the costumer part of logic is covered. The listner code is programmed in the <i>listner.ipynb</i> file in the same directory. This allows the program to function properly.

Firstly, initialize the spark section.

In [8]:
import pandas as pd 

appName = "Twitter Real-time Analytics via Kafka and Spark"
master = "local"
conf = pyspark.SparkConf().set('spark.driver.host','127.0.0.1').setAppName(appName).setMaster(master)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder \
    .appName("Our Twitter Spark and Kafka example") \
        .getOrCreate()

sc

22/10/26 15:19:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


<SparkContext master=local[*] appName=GenericAppName>

Second, initialize the kafka consumer.

In [9]:
from kafka import KafkaConsumer
import json

topic_name = 'mqtt_attack'

kafka_consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='latest',
    enable_auto_commit=True,
    auto_commit_interval_ms =  5000,
    fetch_max_bytes = 128,
    max_poll_records = 100,
    value_deserializer=lambda x: x.decode('utf-8'))

Third, list out the possible attack we are interested in and setup the variables to carry the filtered result of tweeter stream.

In [10]:
df_train.select("target").distinct().show()
attacks = [
    "slowite",
    "bruteforce",
    "flood",
    "malformed",
    "dos"
]

from pyspark.sql.functions import *
from pyspark.sql.types import *

streamed_data = None
res = None
i = 0

+----------+
|    target|
+----------+
|   slowite|
|bruteforce|
|     flood|
| malformed|
|       dos|
|legitimate|
+----------+



Parse the messages, one at a time to avoid huge dataframe. Moreover, we process 10 msg in each cell execution, in order to start or stop the iteration more easily. 

For example, with 5 minites tweeter stream, we got 48 message items. We only need to run the cell 5 times and mannually feed in 2 line of irrelevent messages in the producer to end the for loop in a proper way (with out interrupting the kernel).

In [17]:
start = True
for message in kafka_consumer:
    # I prrocess 10 msg each round
    if i%10==0 and not start:   
        print(f"{'#'*10} MSG {i} {'#'*10}")
        break
    else:
        start = False
    i = i + 1
    # Read the tweet from the topic
    data = [message.value]
    tweet = spark.createDataFrame(data,"string")

    if streamed_data is None:
        streamed_data = tweet.withColumn('word', explode(split(col('value'), ' '))) \
                    .groupBy('word') \
                    .count() \
                    .sort('count', ascending=False)
        
        res = streamed_data.filter(
            col('word').like(attacks[0]) |
            col('word').like(attacks[1]) |
            col('word').like(attacks[2]) |
            col('word').like(attacks[3]) |
            col('word').like(attacks[4]) 
        )
            
    else:
        streamed_data = tweet.withColumn('word', explode(split(col('value'), ' '))) \
                    .groupBy('word') \
                    .count() \
                    .sort('count', ascending=False)

        res = res.union(
            streamed_data.filter(
                col('word').like(attacks[0]) |
                col('word').like(attacks[1]) |
                col('word').like(attacks[2]) |
                col('word').like(attacks[3]) |
                col('word').like(attacks[4])
            )
        )


########## MSG 40 ##########


In the end, we found no mention of the attack of interest. So we would conclude that none of the listed target seems to be popular under the context of IoT system.

Note that our listener's filter constraints / stream rules are

```
IoT-attack
IoT-crime
```

In [18]:
res.show()

+----+-----+
|word|count|
+----+-----+
+----+-----+

